In [1]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result




In [7]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(.4)
    return(nba_season)

nba_season = get_all_games(team_ids,s)
nba_season = nba_season.sort_values(by=['GAME_DATE','GAME_ID'])
nba_season.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
81,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,126,46,82,0.561,12,35,0.343,22,28,0.786,6,30,36,24,8,3,10,24,9.0
81,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0
81,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,45,99,0.455,16,45,0.356,17,23,0.739,11,37,48,31,11,4,18,23,14.0
81,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,40,94,0.426,10,40,0.250,19,25,0.760,9,39,48,23,12,4,21,18,-14.0
81,22022,1610612753,ORL,Orlando Magic,0022200003,2022-10-19,ORL @ DET,L,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,-4.0


In [8]:
home_team = nba_season[nba_season.MATCHUP.str.contains(' vs. ')]
away_team = nba_season[nba_season.MATCHUP.str.contains(' @ ')]

home_team_1 = home_team[['SEASON_ID','GAME_ID','GAME_DATE','TEAM_ID','TEAM_ABBREVIATION','MATCHUP','WL']]
home_team_1= pd.get_dummies(home_team_1, columns=['TEAM_ABBREVIATION','WL'], prefix=['HOME','HOME'])
home_team_1 = home_team_1.drop(['HOME_L'],axis=1)
home_team_1 = home_team_1.rename(columns={'TEAM_ID':'HOME_TEAM_ID','HOME_W':'HomeWIN = 1'})


away_team_1 = away_team[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION']]
away_team_1 =  pd.get_dummies(away_team_1, columns=['TEAM_ABBREVIATION'], prefix=["AWAY"])

In [ ]:
#Get home team game stats
from nba_api.stats.endpoints import teamdashboardbyopponent

call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id=1610612757,date_to_nullable='2022-12-12')
overall_team_dashboard = call.get_data_frames()[3]
#conference_team_dashboard = call.get_data_frames()[1]
#division_team_dashboard = call.get_data_frames()[2]
#opponent_team_dashboard = call.get_data_frames()[3]

overall_team_dashboard.head()
#conference_team_dashboard.head()
#division_team_dashboard.head()
#opponent_team_dashboard.head()

In [11]:
'''function to get team stats per game and vs_Opponent stats per game for each home team'''
from nba_api.stats.endpoints import teamdashboardbyopponent
#create a tuple list of game dates and home team id's to pass to the api call
gameDate_homeTeam = list(zip(home_team_1.GAME_DATE, home_team_1.HOME_TEAM_ID))

def team_stats_pergame(gameDate_Team):
    #create an empty dataframe to hold the results
    teamOverall_stats = pd.DataFrame()
    #create an empty dataframe to hold vs_Opponent results
    teamOpponent_stats = pd.DataFrame()

    #for row in gameDate_homeTeam:
    for tuple_item in gameDate_Team:
        #assign the game date and team id to variables to pass to the api call as parameters called dt and team_id
        dt = tuple_item[0]  
        team_id = tuple_item[1]
        #call the api and pass the parameters
        call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id= team_id,date_to_nullable= dt)
        #get the data frame from the api call
        overall_data = call.get_data_frames()[0]
        #get the vs_Opponent data frame from the api call
        opponent_data = call.get_data_frames()[3]
        #concat the overall dataframe into the teamOverall_stats dataframe
        teamOverall_stats = pd.concat([teamOverall_stats,overall_data])
        #concat the vs_Opponent dataframe into the teamOpponent_stats dataframe
        teamOpponent_stats = pd.concat([teamOpponent_stats,opponent_data])
        #sleep for 1 second to avoid api call limit
        time.sleep(.4)
    return(teamOverall_stats,teamOpponent_stats)

#call the function and pass the gameDate_homeTeam list
team_Stats,Opp_stats = team_stats_pergame(gameDate_homeTeam)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [58]:
#add _team to the column names of the team_Stats dataframe
team_Stats.columns = [str(col) + '_team' for col in team_Stats.columns]
#add _Opp to the column names of the Opp_stats dataframe
Opp_stats.columns = [str(col) + '_Opp' for col in Opp_stats.columns]
#reset the index of the team_Stats dataframe
team_Stats = team_Stats.reset_index(drop=True)
#reset the index of the Opp_stats dataframe
Opp_stats = Opp_stats.reset_index(drop=True)
#reset the index of the homeTeam1_10 dataframe
homeTeam1_10 = homeTeam1_10.reset_index(drop=True)
#get the column names of all three dataframes and combine them into a list
team_Stats_cols = team_Stats.columns.tolist()
Opp_stats_cols = Opp_stats.columns.tolist()
homeTeam1_10_cols = homeTeam1_10.columns.tolist()
#combine the three lists into one list
cols = homeTeam1_10_cols + team_Stats_cols + Opp_stats_cols
#concat the team_Stats,Opp_stats and homeTeam1_10 dataframes on column axis to create the final dataframe
homeTeam_full = pd.concat([homeTeam1_10,team_Stats,Opp_stats],axis=1,ignore_index=True)
#assign the column names to the final dataframe
homeTeam_full.columns = cols

In [45]:
#get fist 10 rows of home_team_1
homeTeam1_10 = home_team_1.head(10)